#Assignment 7 
  #1. Mount the drive
  #2. Append the drive path to system path
  #3. Import files/modules and execute modules

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!ls /content/gdrive/My\ Drive/eva-04/S7/*.py

'/content/gdrive/My Drive/eva-04/S7/CIFARModel.py'
'/content/gdrive/My Drive/eva-04/S7/collect_env.py'
'/content/gdrive/My Drive/eva-04/S7/global_fun.py'
'/content/gdrive/My Drive/eva-04/S7/__init__.py'
'/content/gdrive/My Drive/eva-04/S7/Model.py'


# New Section

In [0]:
#!cat '/content/gdrive/My Drive/eva-04/S7/CIFARModel.py'

In [4]:
!cat '/content/gdrive/My Drive/eva-04/S7/global_fun.py'

from __future__ import print_function
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import copy
from torchsummary import summary
from torchvision import datasets, transforms
import Model

def train_model(model, device, train_loader, optimizer, epoch,train_losses,train_acc, doL1 = 0,doL2 = 0,LAMBDA = 0):
  print('L1=',doL1,';L2=',doL2,';LAMBDA=',LAMBDA,'epoch=',epoch)
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you

We'll need to update our path to import from Drive.

In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/eva-04/S7/')

Now we can import the library and use the function.

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
def miscImages(model, device, test_loader):
    model.eval()
    test_loss = 0
    incorrect = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

            for i in range(len(target)):
              if pred[i].item() != target[i]:
                incorrect += 1
                print('\n\n{} [ Predicted Value: {}, Actual Value: {} ]'.format(
                incorrect, pred[i].item(), target[i], ))
                plt.imshow(data[i].cpu().numpy().squeeze(), cmap='gray_r')
                plt.show()


In [0]:

import matplotlib.pyplot as plt
def draw_accuracy_loss_change_graps(model_0,model_l1,model_l2,model_l1_l2):
    fig, axs = plt.subplots(2,2,figsize=(30,20))
    #print('train_losses=',len(train_losses))
    #print('test_losses=',len(test_losses))

    axs[0,0].plot(model_0.m_test_losses,color='black',label='No Regularization')
    axs[0,0].plot(model_l1.m_test_losses,color='red',label='L1 Regularization')
    axs[0,0].plot(model_l2.m_test_losses,color='blue',label='L2 Regularization')
    axs[0,0].plot(model_l1_l2.m_test_losses,color='green',label='Both L1 and L2 Regularization')
    axs[0,0].set_title("Validation Loss Change")
    axs[0,0].legend(loc="center")

    axs[0,1].plot(model_0.m_test_acc,color='black',label='No Regularization')
    axs[0,1].plot(model_l1.m_test_acc,color='red',label='L1 Regularization')
    axs[0,1].plot(model_l2.m_test_acc,color='blue',label='L2 Regularization')
    axs[0,1].plot(model_l1_l2.m_test_acc,color='green',label='Both L1 and L2 Regularization')
    axs[0,1].set_title("Validation Accuracy Change")
    axs[0,1].legend(loc="center")

    axs[1,0].plot(model_0.m_train_losses,color='black',label='No Regularization')
    axs[1,0].plot(model_l1.m_train_losses,color='red',label='L1 Regularization')
    axs[1,0].plot(model_l2.m_train_losses,color='blue',label='L2 Regularization')
    axs[1,0].plot(model_l1_l2.m_train_losses,color='green',label='Both L1 and L2 Regularization')
    axs[1,0].set_title("Training Loss Change")
    axs[1,0].legend(loc="center")

    axs[1,1].plot(model_0.m_train_acc,color='black',label='No Regularization')
    axs[1,1].plot(model_l1.m_train_acc,color='red',label='L1 Regularization')
    axs[1,1].plot(model_l2.m_train_acc,color='blue',label='L2 Regularization')
    axs[1,1].plot(model_l1_l2.m_train_acc,color='green',label='Both L1 and L2 Regularization')
    axs[1,1].set_title("Training Accuracy Change")
    axs[1,1].legend(loc="center")

In [9]:
import Model as m
import CIFARModel as cifar

import importlib
importlib.reload(m)
importlib.reload(cifar)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = m.CIFAR_Depthwise_Separable_Model().to(device)

SEED = 1
# CUDA?
print("CUDA Available?", use_cuda)

# For reproducibility
torch.manual_seed(SEED)

if use_cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
#dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if use_cuda else dict(shuffle=True, batch_size=64)

# train dataloader
#train_loader = torch.utils.data.DataLoader(g_train, **dataloader_args)

# test dataloader
#test_loader = torch.utils.data.DataLoader(g_test, **dataloader_args)

cifar_model_l1_l2 = cifar.CIFARModel(model)

from global_fun import test_model
from global_fun import train_model
from global_fun import show_summary, run_model 


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
cuda
CUDA Available? True


RUNNING WITH REGULARIZATION

In [10]:

run_model(cifar_model_l1_l2, device, doL1= 1, doL2=1, LAMBDA=0.0001, EPOCHS=30)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 1 ;L2= 1 ;LAMBDA= 0.0001 epoch= 0


Loss=3.3372340202331543 Batch_id=195 Accuracy=45.49: 100%|██████████| 196/196 [01:11<00:00,  2.87it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0070, Accuracy: 4018/10000 (40.18%)

EPOCH: 1
L1= 1 ;L2= 1 ;LAMBDA= 0.0001 epoch= 1


Loss=3.859692096710205 Batch_id=195 Accuracy=61.92: 100%|██████████| 196/196 [01:12<00:00,  2.84it/s]
  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0035, Accuracy: 7334/10000 (73.34%)

EPOCH: 28
L1= 1 ;L2= 1 ;LAMBDA= 0.0001 epoch= 28


Loss=19.88888168334961 Batch_id=195 Accuracy=92.80: 100%|██████████| 196/196 [01:14<00:00,  2.75it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0037, Accuracy: 7174/10000 (71.74%)

EPOCH: 29
L1= 1 ;L2= 1 ;LAMBDA= 0.0001 epoch= 29


Loss=20.33182144165039 Batch_id=195 Accuracy=93.11: 100%|██████████| 196/196 [01:17<00:00,  2.82it/s]



Test set: Average loss: 0.0044, Accuracy: 7033/10000 (70.33%)



In [0]:
run_model(cifar_model_l1_l2, device, doL1= 1, doL2=1, LAMBDA=0.0001, EPOCHS=20)

DROPOUT INCREASED TO 0.2

In [12]:
model2 = m.CIFAR_Depthwise_Separable_Model(dropout_value = 0.2).to(device)
cifar_model_drop = cifar.CIFARModel(model2)
run_model(cifar_model_drop, device, doL1= 1, doL2=1, LAMBDA=0.00001, EPOCHS=20)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 0


Loss=1.4323710203170776 Batch_id=195 Accuracy=43.72: 100%|██████████| 196/196 [01:12<00:00,  2.82it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0072, Accuracy: 3718/10000 (37.18%)

EPOCH: 1
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 1


Loss=1.4480725526809692 Batch_id=195 Accuracy=61.06: 100%|██████████| 196/196 [01:12<00:00,  2.89it/s]
  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0038, Accuracy: 6674/10000 (66.74%)

EPOCH: 18
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 18


Loss=1.675093412399292 Batch_id=195 Accuracy=84.62: 100%|██████████| 196/196 [01:10<00:00,  2.89it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0042, Accuracy: 6284/10000 (62.84%)

EPOCH: 19
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 19


Loss=1.9033282995224 Batch_id=195 Accuracy=85.51: 100%|██████████| 196/196 [01:11<00:00,  2.89it/s]



Test set: Average loss: 0.0034, Accuracy: 7076/10000 (70.76%)



In [0]:
model2 = m.CIFAR_Depthwise_Separable_Model(dropout_value = 0.2).to(device)
cifar_model_drop = cifar.CIFARModel(model2)
run_model(cifar_model_drop, device, doL1= 1, doL2=1, LAMBDA=0.00001, EPOCHS=20)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 0


Loss=1.4323710203170776 Batch_id=195 Accuracy=43.72: 100%|██████████| 196/196 [01:12<00:00,  2.82it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0072, Accuracy: 3718/10000 (37.18%)

EPOCH: 1
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 1


Loss=1.4480725526809692 Batch_id=195 Accuracy=61.06: 100%|██████████| 196/196 [01:12<00:00,  2.89it/s]
  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0038, Accuracy: 6674/10000 (66.74%)

EPOCH: 18
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 18


Loss=1.675093412399292 Batch_id=195 Accuracy=84.62: 100%|██████████| 196/196 [01:10<00:00,  2.89it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0042, Accuracy: 6284/10000 (62.84%)

EPOCH: 19
L1= 1 ;L2= 1 ;LAMBDA= 1e-05 epoch= 19


Loss=1.9033282995224 Batch_id=195 Accuracy=85.51: 100%|██████████| 196/196 [01:11<00:00,  2.89it/s]



Test set: Average loss: 0.0034, Accuracy: 7076/10000 (70.76%)



In [13]:
run_model(cifar_model_drop, device, doL1= 1, doL2=0, LAMBDA=0.1, EPOCHS=20)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 1 ;L2= 0 ;LAMBDA= 0.1 epoch= 0


  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0046, Accuracy: 6017/10000 (60.17%)

EPOCH: 1
L1= 1 ;L2= 0 ;LAMBDA= 0.1 epoch= 1


  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0044, Accuracy: 6419/10000 (64.19%)

EPOCH: 18
L1= 1 ;L2= 0 ;LAMBDA= 0.1 epoch= 18


Loss=12003.4169921875 Batch_id=195 Accuracy=91.26: 100%|██████████| 196/196 [01:10<00:00,  2.90it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0036, Accuracy: 6942/10000 (69.42%)

EPOCH: 19
L1= 1 ;L2= 0 ;LAMBDA= 0.1 epoch= 19


Loss=12165.9052734375 Batch_id=195 Accuracy=91.20: 100%|██████████| 196/196 [01:10<00:00,  2.90it/s]



Test set: Average loss: 0.0033, Accuracy: 7279/10000 (72.79%)



Dropout increased to 0.2


In [14]:
run_model(cifar_model_drop, device, doL1= 1, doL2=1, LAMBDA=0.1, EPOCHS=20)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 1 ;L2= 1 ;LAMBDA= 0.1 epoch= 0


  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0039, Accuracy: 6900/10000 (69.00%)

EPOCH: 1
L1= 1 ;L2= 1 ;LAMBDA= 0.1 epoch= 1


  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0038, Accuracy: 6978/10000 (69.78%)

EPOCH: 18
L1= 1 ;L2= 1 ;LAMBDA= 0.1 epoch= 18


  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0036, Accuracy: 7157/10000 (71.57%)

EPOCH: 19
L1= 1 ;L2= 1 ;LAMBDA= 0.1 epoch= 19


Loss=35060.5234375 Batch_id=195 Accuracy=94.10: 100%|██████████| 196/196 [01:11<00:00,  2.87it/s]



Test set: Average loss: 0.0045, Accuracy: 6655/10000 (66.55%)



In [0]:
run_model(cifar_model_drop, device, doL1= 0, doL2=0, LAMBDA=0, EPOCHS=20)

In [15]:
run_model(cifar_model_drop, device, doL1= 0, doL2=0, LAMBDA=0, EPOCHS=20)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 0


Loss=0.20363588631153107 Batch_id=195 Accuracy=93.80: 100%|██████████| 196/196 [01:10<00:00,  2.92it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0044, Accuracy: 6609/10000 (66.09%)

EPOCH: 1
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 1


Loss=0.13072840869426727 Batch_id=195 Accuracy=94.06: 100%|██████████| 196/196 [01:10<00:00,  2.94it/s]
  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0039, Accuracy: 7137/10000 (71.37%)

EPOCH: 18
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 18


Loss=0.19308339059352875 Batch_id=195 Accuracy=95.07: 100%|██████████| 196/196 [01:10<00:00,  2.91it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0044, Accuracy: 6929/10000 (69.29%)

EPOCH: 19
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 19


Loss=0.1896900236606598 Batch_id=195 Accuracy=94.70: 100%|██████████| 196/196 [01:10<00:00,  2.92it/s]



Test set: Average loss: 0.0037, Accuracy: 7205/10000 (72.05%)



In [16]:
run_model(cifar_model_drop, device, doL1= 0, doL2=0, LAMBDA=0, EPOCHS=40)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 0


Loss=0.11422179639339447 Batch_id=195 Accuracy=94.92: 100%|██████████| 196/196 [01:10<00:00,  2.91it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0042, Accuracy: 7059/10000 (70.59%)

EPOCH: 1
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 1


Loss=0.12538889050483704 Batch_id=195 Accuracy=95.15: 100%|██████████| 196/196 [01:10<00:00,  2.92it/s]
  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0042, Accuracy: 7116/10000 (71.16%)

EPOCH: 38
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 38


Loss=0.11534328758716583 Batch_id=195 Accuracy=96.00: 100%|██████████| 196/196 [01:10<00:00,  2.91it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0040, Accuracy: 7211/10000 (72.11%)

EPOCH: 39
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 39


Loss=0.09426665306091309 Batch_id=195 Accuracy=96.40: 100%|██████████| 196/196 [01:10<00:00,  2.94it/s]



Test set: Average loss: 0.0041, Accuracy: 7221/10000 (72.21%)



In [17]:
run_model(cifar_model_drop, device, doL1= 0, doL2=0, LAMBDA=0, EPOCHS=40)

  0%|          | 0/196 [00:00<?, ?it/s]

EPOCH: 0
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 0


Loss=0.14198610186576843 Batch_id=195 Accuracy=96.43: 100%|██████████| 196/196 [01:11<00:00,  2.92it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0039, Accuracy: 7335/10000 (73.35%)

EPOCH: 1
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 1


Loss=0.10932661592960358 Batch_id=195 Accuracy=96.21: 100%|██████████| 196/196 [01:10<00:00,  2.93it/s]
  0%|          | 0/196 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0042, Accuracy: 7314/10000 (73.14%)

EPOCH: 38
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 38


Loss=0.1142563596367836 Batch_id=195 Accuracy=96.79: 100%|██████████| 196/196 [01:10<00:00,  2.92it/s]
  0%|          | 0/196 [00:00<?, ?it/s]


Test set: Average loss: 0.0044, Accuracy: 7150/10000 (71.50%)

EPOCH: 39
L1= 0 ;L2= 0 ;LAMBDA= 0 epoch= 39


Loss=0.07670222222805023 Batch_id=195 Accuracy=97.01: 100%|██████████| 196/196 [01:10<00:00,  2.92it/s]



Test set: Average loss: 0.0043, Accuracy: 7211/10000 (72.11%)

